#### 对函数y=2xTx    关于列向量x求导

In [1]:
import torch
x=torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

#### 计算y关于x的梯度之前，需要一个地方来存储梯度

In [3]:
x.requires_grad_(True) # 等价于x=torch.arange(4.0,required_gred=True)
x.grad # defalut None

#### 计算y

In [6]:
y=2*torch.dot(x,x)     # y是标量
y

tensor(28., grad_fn=<MulBackward0>)

#### 通过调用反向传播函数来自动计算y关于x每个分量的梯度

In [7]:
y.backward()
x.grad == 4*x

tensor([True, True, True, True])

#### 计算x的另一个函数

In [8]:
# 默认情况下，PyTorch会累计梯度，需要清楚之前的值
x.grad.zero_()  #下划线表示函数重写内容
y=x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

#### 深度学习中，目的不是计算微分矩阵，而是批量中每个样本单独计算的偏导数之和

In [9]:
# 对非标量调用backward需要传入一个gradient参数
x.grad.zero_()
y=x*x   # 很少对向量的函数求导，大部分情况是对标量求导
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

#### 将某些计算移动到记录的计算图之外
##### 需要将参数固定的时候很有用

In [10]:
x.grad.zero_()
y=x*x
u=y.detach()  # 把u当做一个常数，不在计算图之内  
z=u*x

z.sum().backward()
x.grad==u

tensor([True, True, True, True])

In [11]:
x.grad.zero_()    # y还是在计算图内的，被当做关于x的变量来计算
y.sum().backward()
x.grad==2*x

tensor([True, True, True, True])

#### 即使构建函数的计算图需要通过Python控制流(例如，条件、循环或任意函数调用)，可以计算得到变量的梯度

In [14]:
def f(a):
    b=a*2
    while b.norm()<1000:
        b=b*2
    if b.sum()>0:
        c=b
    else:
        c=100*b
    return c
a=torch.randn(size=(),requires_grad=True)
d=f(a)
a.grad==d/a

False